In [1]:
from rdkit.Chem import AllChem
from scipy import stats
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns
import time
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [2]:
import pandas as pd
import numpy as np

In [3]:
broad_data = pd.read_csv("./broad_data_filtered_89_inchi.csv") 
assay_list= broad_data.columns.to_list()[2:91]
assay_data= broad_data.iloc[:, :91]
assay_data = assay_data.drop(columns= ["smiles_r"])
assay_data

,InChICode_standardised,178_684,180_686,183_604,184_606,185_665,185_666,186_668,190_628,190_629,...,273_751,274_752,275_753,275_754,276_755,278_757,279_758,280_759,283_762,93_259
0,InChI=1S/C20H22N2O3S2/c1-3-25-19(23)15-4-8-16(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,InChI=1S/C15H13N3O3/c19-13(8-12-14(20)18-15(21...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
2,InChI=1S/C20H22N4/c1-15(16-9-3-2-4-10-16)21-19...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
3,InChI=1S/C21H21BrN4O3S/c1-4-5-16(29-13-8-6-12(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,InChI=1S/C20H17FN2O5/c1-11-17(18(23-20(25)22-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16165,InChI=1S/2C24H28N2O4/c2*1-3-7-17-10-11-20-21-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN
16166,InChI=1S/2C28H35N3O3/c2*1-3-7-21-10-11-24-25-2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
16167,InChI=1S/2C28H35N3O3/c2*1-3-7-21-10-11-24-25-2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN
16168,InChI=1S/2C28H29N5O4/c2*1-3-7-19-10-11-23-24-2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN


In [4]:
CP_features = pd.read_csv("./CP_filtered_16170.csv") 
CP_features_list= CP_features.columns.to_list()[:-1]
#CP_features = CP_features.to_numpy()
CP_features

,Cells_Correlation_Correlation_RNA_AGP,Cells_Correlation_Costes_AGP_RNA,Cells_Correlation_K_AGP_DNA,Cells_Correlation_K_DNA_AGP,Cells_Correlation_K_DNA_ER,Cells_Correlation_K_DNA_Mito,Cells_Correlation_K_ER_DNA,Cells_Correlation_K_ER_Mito,Cells_Correlation_K_Mito_DNA,Cells_Correlation_K_RNA_DNA,...,Nuclei_Texture_SumVariance_AGP_10_0,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,InChICode_standardised
0,-0.157794,-0.099680,-0.164429,0.070906,0.036844,0.208466,-0.055008,0.126274,-0.202764,-0.119896,...,-0.165474,-0.145238,-0.255324,-0.183818,-0.294134,-0.011429,-0.017739,-0.067972,-0.218854,InChI=1S/C20H22N2O3S2/c1-3-25-19(23)15-4-8-16(...
1,-0.054050,-0.057673,-0.208181,0.059523,0.066208,0.133907,-0.112603,0.081640,-0.244245,-0.154735,...,-0.111427,-0.645817,-0.216536,0.119748,-0.181693,0.020779,-0.000956,0.147834,-0.127996,InChI=1S/C15H13N3O3/c19-13(8-12-14(20)18-15(21...
2,-0.119915,-0.060212,-0.379775,0.133999,0.231516,0.350750,-0.177105,0.068798,-0.344051,-0.353201,...,-0.528930,-0.401087,-0.502001,-0.533541,-0.477806,-0.256362,-0.102372,-0.214411,-0.261781,InChI=1S/C20H22N4/c1-15(16-9-3-2-4-10-16)21-19...
3,-0.087385,-0.101862,-0.074942,0.026228,0.099839,0.044025,-0.111699,-0.032776,-0.076653,-0.086310,...,-0.101392,-0.502601,-0.440860,-1.100426,-0.090082,-0.066900,-0.097931,-0.332383,0.047669,InChI=1S/C21H21BrN4O3S/c1-4-5-16(29-13-8-6-12(...
4,0.026187,-0.037733,-0.210272,0.197219,0.232726,0.412734,-0.112595,0.149297,-0.262999,-0.163375,...,0.282515,-0.239423,0.522684,0.065040,-0.293402,0.113185,0.298401,0.059659,-0.232806,InChI=1S/C20H17FN2O5/c1-11-17(18(23-20(25)22-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16165,0.004377,-0.071760,0.038217,-0.040111,-0.104244,-0.039729,0.049694,0.018068,0.030077,0.055247,...,-0.086669,-0.192219,-0.070905,-0.207405,-0.026708,-0.006933,-0.041496,-0.076398,-0.006674,InChI=1S/2C24H28N2O4/c2*1-3-7-17-10-11-20-21-1...
16166,-0.026166,-0.049399,0.002118,0.003419,0.012760,0.014354,-0.006235,-0.002326,-0.026698,0.006378,...,-0.101214,-0.413595,-0.242339,0.147337,-0.129706,0.001940,-0.062467,0.014129,-0.048734,InChI=1S/2C28H35N3O3/c2*1-3-7-21-10-11-24-25-2...
16167,0.006194,-0.142756,-0.005567,-0.008616,0.006643,0.021155,-0.009634,0.049145,-0.046749,0.015002,...,0.011103,0.242727,0.135037,-0.260295,-0.061008,0.011743,-0.008725,-0.038521,-0.030230,InChI=1S/2C28H35N3O3/c2*1-3-7-21-10-11-24-25-2...
16168,0.014116,-0.061477,-0.037600,0.021552,-0.017481,0.042454,0.019394,0.043665,-0.044074,-0.008478,...,-0.034148,0.422416,0.071783,-0.026039,-0.097359,-0.011104,-0.123216,0.021086,-0.021521,InChI=1S/2C28H29N5O4/c2*1-3-7-19-10-11-23-24-2...


In [5]:
mfp_features = pd.read_csv("./Mfp_filtered_16170.csv") 
mfp_features_list = mfp_features.columns.to_list()[:-1]
mfp_features['fp'] = [(x) for x in mfp_features.iloc[:, :-1].to_numpy()]
mfp_features = mfp_features[["InChICode_standardised", "fp"]]
mfp_features

,InChICode_standardised,fp
0,InChI=1S/C20H22N2O3S2/c1-3-25-19(23)15-4-8-16(...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C15H13N3O3/c19-13(8-12-14(20)18-15(21...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C20H22N4/c1-15(16-9-3-2-4-10-16)21-19...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C21H21BrN4O3S/c1-4-5-16(29-13-8-6-12(...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,InChI=1S/C20H17FN2O5/c1-11-17(18(23-20(25)22-1...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
...,...,...
16165,InChI=1S/2C24H28N2O4/c2*1-3-7-17-10-11-20-21-1...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
16166,InChI=1S/2C28H35N3O3/c2*1-3-7-21-10-11-24-25-2...,"[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
16167,InChI=1S/2C28H35N3O3/c2*1-3-7-21-10-11-24-25-2...,"[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
16168,InChI=1S/2C28H29N5O4/c2*1-3-7-19-10-11-23-24-2...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [6]:
broad_data= pd.merge(assay_data, CP_features)
broad_data= pd.merge(broad_data, mfp_features)
broad_data

,InChICode_standardised,178_684,180_686,183_604,184_606,185_665,185_666,186_668,190_628,190_629,...,Nuclei_Texture_SumVariance_AGP_10_0,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,fp
0,InChI=1S/C20H22N2O3S2/c1-3-25-19(23)15-4-8-16(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.165474,-0.145238,-0.255324,-0.183818,-0.294134,-0.011429,-0.017739,-0.067972,-0.218854,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C15H13N3O3/c19-13(8-12-14(20)18-15(21...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,-0.111427,-0.645817,-0.216536,0.119748,-0.181693,0.020779,-0.000956,0.147834,-0.127996,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C20H22N4/c1-15(16-9-3-2-4-10-16)21-19...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.528930,-0.401087,-0.502001,-0.533541,-0.477806,-0.256362,-0.102372,-0.214411,-0.261781,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C21H21BrN4O3S/c1-4-5-16(29-13-8-6-12(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.101392,-0.502601,-0.440860,-1.100426,-0.090082,-0.066900,-0.097931,-0.332383,0.047669,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,InChI=1S/C20H17FN2O5/c1-11-17(18(23-20(25)22-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.282515,-0.239423,0.522684,0.065040,-0.293402,0.113185,0.298401,0.059659,-0.232806,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16165,InChI=1S/2C24H28N2O4/c2*1-3-7-17-10-11-20-21-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.086669,-0.192219,-0.070905,-0.207405,-0.026708,-0.006933,-0.041496,-0.076398,-0.006674,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
16166,InChI=1S/2C28H35N3O3/c2*1-3-7-21-10-11-24-25-2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.101214,-0.413595,-0.242339,0.147337,-0.129706,0.001940,-0.062467,0.014129,-0.048734,"[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
16167,InChI=1S/2C28H35N3O3/c2*1-3-7-21-10-11-24-25-2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.011103,0.242727,0.135037,-0.260295,-0.061008,0.011743,-0.008725,-0.038521,-0.030230,"[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
16168,InChI=1S/2C28H29N5O4/c2*1-3-7-19-10-11-23-24-2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.034148,0.422416,0.071783,-0.026039,-0.097359,-0.011104,-0.123216,0.021086,-0.021521,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [7]:
def TanimotoSimilarity(ex1, ex2, attrs):
  """
    >>> v1 = [0,1,0,1]
    >>> v2 = [1,0,1,0]
    >>> TanimotoDist(v1,v2,range(4))
    1.0
    """
  inter = 0.0
  unin = 0.0
  for i in attrs:
    if (ex1[i] or ex2[i]):
      unin += 1
      if (ex1[i] and ex2[i]):
        inter += 1
  if (unin != 0.0):
    return (inter / unin)
  else:
    return 0

In [8]:
def BulkTanimoto(siv1, sivs):
    res = []
    for i in range(len(sivs)): 
        siv2 = sivs[i]
        simVal = TanimotoSimilarity(siv1, siv2, range(len(siv1)))
        res.append(simVal)
    return res

In [9]:
#Find similarity between each test compounds and train set compounds of same class and 
#store the lowest tanimoto similarity of MFP keys
neighbours_limit = 5
def findsimscore(assay, train, test): 
    ts_nearest_list = []

    for index, row in test.iterrows():

        fps_test = row["fp"]

        compound_activity= row[assay]
        #print(compound_activity)
        temp_train = pd.DataFrame()
        temp_train= train[train[assay] == compound_activity]
        #print(temp.shape)

        fps_temp_train=[(x) for x in temp_train["fp"]]
        ts_nearest = []
        #  median of 5 most similar compounds by sorted assay in descending order
        ts_nearest = (np.sort(BulkTanimoto(fps_test ,fps_temp_train))[::-1])[:neighbours_limit]
        ts_nearest=np.median(ts_nearest)
        ts_nearest_list.append(ts_nearest)

    ts_nearest_list=np.array(ts_nearest_list)
    ts_nearest_list= ts_nearest_list.reshape(len(test))

    df_ts_nearest = pd.DataFrame({ "ts_nearest": ts_nearest_list})
    return df_ts_nearest

In [10]:
#Find similarity between each test compounds and train set compounds of same class and 
#
neighbours_limit = 5
def findpearsonscore(assay, train, test): 
    pc_highest_list = []

    for index, row in test.iterrows():

        fps_test = row[CP_features_list].to_numpy()

        compound_activity= row[assay]
        #print(compound_activity)
        temp_train = pd.DataFrame()
        temp_train= train[train[assay] == compound_activity]
        #print(temp.shape)

        fps_temp_train=[(x) for x in temp_train[CP_features_list].to_numpy()]
        pearson_s_list = []
        
        for n in range(len(fps_temp_train)):
            
            pearson= stats.pearsonr(fps_test, fps_temp_train[n])
            pearson_s_list.append(pearson[0])
            #median of 15 positive correlations
            
        pc_highest= np.sort(pearson_s_list)[len(fps_temp_train)-neighbours_limit:]
        pc_lowest= np.sort(pearson_s_list)[:neighbours_limit]    
        
        pc_highest=(np.median(pc_highest))#+ abs(np.median(pc_lowest))) /2
        pc_highest_list.append(pc_highest)
    
    pc_highest_list=np.array(pc_highest_list)
    pc_highest_list= pc_highest_list.reshape(len(test))

    df_pc_highest = pd.DataFrame({ "pc_highest": pc_highest_list})
    return df_pc_highest

In [11]:
def plot_hist_ts(df):
    
    #f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
    
    mean=df['ts_nearest'].mean()
    median=df['ts_nearest'].median()
    mode=df['ts_nearest'].mode().tolist()[0]
    
    
    #sns.boxplot(x=df["td_nearest"], ax=ax_box)
    #ax_box.axvline(mean, color='r', linestyle='--',  label='Mean')
    #ax_box.axvline(median, color='g', linestyle='-',  label='Median')
    #ax_box.axvline(mode, color='b', linestyle='-',  label='Mode')

    #sns.histplot(x=df["td_nearest"],  kde=True, ax=ax_hist)
    #ax_hist.axvline(mean, color='r', linestyle='--', label='Mean')
    #ax_hist.axvline(median, color='g', linestyle='-', label='Median')
    #ax_hist.axvline(mode, color='b', linestyle='-', label='Mode')

    #plt.legend()
    #ax_box.set(xlabel='')
    #plt.show()
    
    return mean, median, mode

def plot_hist_pc(df):
    
    #f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
    
    mean=df['pc_highest'].mean()
    median=df['pc_highest'].median()
    mode=df['pc_highest'].mode().tolist()[0]
    
    
    #sns.boxplot(x=df["pc_highest"], ax=ax_box)
    #ax_box.axvline(mean, color='r', linestyle='--',  label='Mean')
    #ax_box.axvline(median, color='g', linestyle='-',  label='Median')
    #ax_box.axvline(mode, color='b', linestyle='-',  label='Mode')

    #sns.histplot(x=df["pc_highest"],  kde=True, ax=ax_hist)
    #ax_hist.axvline(mean, color='r', linestyle='--', label='Mean')
    #ax_hist.axvline(median, color='g', linestyle='-', label='Median')
    #ax_hist.axvline(mode, color='b', linestyle='-', label='Mode')

    #plt.legend()
    #ax_box.set(xlabel='')
    #plt.show()
    
    return mean, median, mode

In [12]:
#Nested Cross validation
#stat_list_batch: contains assay number, mean, median, mode of tanimoto distance in that cycle, mcc and auc of that cycle
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts
#pred_list_batch contains proba, pred and true values

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler 

from collections import Counter
from numpy import argmax

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from scipy.stats import ks_2samp 
from scipy.stats import randint
from sklearn.model_selection import cross_val_predict

def my_function_nested_rhs(assay, cycle):   
        
    detail_list_batch_train = []

    X_fp = df_assay_train.loc[:, "fp"].to_numpy()
    X_CP = df_assay_train[CP_features_list].to_numpy()
    
    #print(X)
    y = df_assay_train[assay].to_numpy()
    #print(y)
    
    
    #Split
    outercv = StratifiedKFold(n_splits=5,shuffle=True, random_state=cycle)
   
    for i, (train_index, test_index) in enumerate(outercv.split(X_fp, y)):
        
        train= df_assay_train.iloc[train_index]
        test= df_assay_train.iloc[test_index]
        #print(train)
        #print(test)

        df_ts_highest = findsimscore(assay, train, test)
        df_pc_highest = findpearsonscore(assay, train, test)    
        
        mean_ts, median_score, mode_score= plot_hist_ts(df_ts_highest)
        mean_pc, median_score, mode_score= plot_hist_pc(df_pc_highest)
        
        
        #Prepare features
        train_x_fp, test_x_fp, train_y, test_y = list(train.fp), list(test.fp), train[assay], test[assay]
        col_list=[]
      
        train_x_cp=train[CP_features_list].to_numpy()
        test_x_cp=test[CP_features_list].to_numpy()
        
        #MFP
        print("Mfp_Model_Running for assay outercv#", i+1)    
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
         
        param_dist_grid = {
        'max_depth': [10,15,20],
        #'max_features': randint(40, 50),
        'min_samples_leaf': randint(2, 15),
        'min_samples_split': randint(2, 15),
        'n_estimators':[100, 300, 400, 500],
        'bootstrap': [True, False],
        'oob_score': [False],
        'random_state': [42],
        'criterion': ['gini', 'entropy'],
        'n_jobs': [-1],
        'class_weight' : [None, 'balanced']
        }
        
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)

        rsh.fit(train_x_fp, train_y)

        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)                      
        classifier.fit(train_x_fp, train_y)
        
        #Threshold Balancing
        cross_val_prob_fp = cross_val_predict(rsh.best_estimator_, train_x_fp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_fp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_fp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_fp))


        fp_proba = classifier.predict_proba(test_x_fp)[:,1]
        #fp_pred = classifier.predict(test_x_fp) 
        fp_pred  = [ 1 if x>best_thresh_fp  else 0 for x in fp_proba ] 
    
        #Cell Painting
        
        print("CP_Model_Running for assay outercv#", i+1)  
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)
        
        
        rsh.fit(train_x_cp, train_y)
        
        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_    
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)
        classifier.fit(train_x_cp, train_y)
        
        #Threshold Balancing
        cross_val_prob_cp = cross_val_predict(rsh.best_estimator_, train_x_cp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_cp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_cp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_cp))


        CP_proba = classifier.predict_proba(test_x_cp)[:,1]
        #CP_pred = classifier.predict(test_x_cp) 
        CP_pred  = [ 1 if x>best_thresh_cp  else 0 for x in CP_proba ] 
        
        fpr, tpr, _ = roc_curve(test_y, CP_proba)

        #combine results
        for InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, true_val, ts, pc in zip(test.InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, test_y, df_ts_highest.ts_nearest,  df_pc_highest.pc_highest):
            detail_list_batch_train.append([assay, InChICode_standardised, fp_proba, fp_pred,best_thresh_fp, CP_proba, CP_pred, best_thresh_cp, true_val, ts, pc, "Training"])
    
    return detail_list_batch_train

In [13]:
#Predict External Test Set
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts


def my_function_heldout_rhs(assay, cycle):   
        
        detail_list_batch_test = []

        train= df_assay_train
        test= df_assay_test

        df_ts_highest = findsimscore(assay, train, test)
        df_pc_highest = findpearsonscore(assay, train, test)    
        
        mean_ts, median_score, mode_score= plot_hist_ts(df_ts_highest)
        mean_pc, median_score, mode_score= plot_hist_pc(df_pc_highest)
        
        
        #Prepare features
        train_x_fp, test_x_fp, train_y, test_y = list(train.fp), list(test.fp), train[assay], test[assay]
        col_list=[]
      
        train_x_cp=train[CP_features_list].to_numpy()
        test_x_cp=test[CP_features_list].to_numpy()
        
        #MFP
        print("Mfp_Model_Running for assay")    
        inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
         
        param_dist_grid = {
        'max_depth': [10,15,20],
        #'max_features': randint(40, 50),
        'min_samples_leaf': randint(2, 15),
        'min_samples_split': randint(2, 15),
        'n_estimators':[100, 300, 400, 500],
        'bootstrap': [True, False],
        'oob_score': [False],
        'random_state': [42],
        'criterion': ['gini', 'entropy'],
        'n_jobs': [-1],
        'class_weight' : [None, 'balanced']
        }
        
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)

        rsh.fit(train_x_fp, train_y)

        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)                      
        classifier.fit(train_x_fp, train_y)
        
        #Threshold Balancing
        cross_val_prob_fp = cross_val_predict(rsh.best_estimator_, train_x_fp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_fp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_fp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_fp))


        fp_proba = classifier.predict_proba(test_x_fp)[:,1]
        #fp_pred = classifier.predict(test_x_fp) 
        fp_pred  = [ 1 if x>best_thresh_fp  else 0 for x in fp_proba ] 
    
        #Cell Painting
        
        print("CP_Model_Running for assay")  
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)
        
        
        rsh.fit(train_x_cp, train_y)
        
        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_    
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)
        classifier.fit(train_x_cp, train_y)
        
        #Threshold Balancing
        cross_val_prob_cp = cross_val_predict(rsh.best_estimator_, train_x_cp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_cp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_cp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_cp))


        CP_proba = classifier.predict_proba(test_x_cp)[:,1]
        #CP_pred = classifier.predict(test_x_cp) 
        CP_pred  = [ 1 if x>best_thresh_cp  else 0 for x in CP_proba ] 
        
        fpr, tpr, _ = roc_curve(test_y, CP_proba)

        #combine results
        for InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, true_val, ts, pc in zip(test.InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, test_y, df_ts_highest.ts_nearest,  df_pc_highest.pc_highest):
            detail_list_batch_test.append([assay, InChICode_standardised, fp_proba, fp_pred,best_thresh_fp, CP_proba, CP_pred, best_thresh_cp, true_val, ts, pc, "HeldOut"])
    
        return detail_list_batch_test

In [14]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split

master_detail_list=[]
for assay in tqdm(assay_list[:]):
    
    print("Assay ", assay )
    df_assay= broad_data[pd.notnull(broad_data[assay])]

    print("Total Compounds ", len(df_assay) )

    df_assay_inactive = df_assay[df_assay[assay] == 0]
    inactives_n=len(df_assay_inactive)
    print("Inactive Compounds ", inactives_n)

    df_assay_active = df_assay[df_assay[assay] == 1]
    actives_n=len(df_assay_active)
    print("Active Compounds ", actives_n)

    if(inactives_n > 3*actives_n):
        df_assay_inactive= df_assay_inactive.sample(3*actives_n, random_state=1)
        print("Inactive Compounds (after Undersamplimg) ", len(df_assay_inactive) )
        df_assay= df_assay_active.append(df_assay_inactive) 
        print("Total Compounds (after Undersamplimg) ", len(df_assay) )
        
    elif(actives_n > 3*inactives_n):
        df_assay_active= df_assay_active.sample(3*inactives_n, random_state=1)
        print("Active Compounds (after Undersamplimg) ", len(df_assay_active) )
        df_assay= df_assay_inactive.append(df_assay_active) 
        print("Total Compounds (after Undersamplimg) ", len(df_assay) )
        
    else:
        print("No Undersamplimg; Assay already in 1:3 ratio or less", len(df_assay) )
        
        
    df_assay= df_assay.reset_index(drop=True)
    
    df_assay_train, df_assay_test= train_test_split(df_assay, stratify=df_assay[[assay]], test_size=0.20, random_state=42)
    
    print("Perform nested cross validation on training set")
    detail_list_train =  my_function_nested_rhs(assay, 42)
   
    
    print("Predictions on held out set")
    detail_list_test = my_function_heldout_rhs(assay, 42)

    
    master_detail_list.append(detail_list_train) 
    master_detail_list.append(detail_list_test) 
    
    

  0%|                                                    | 0/89 [00:00<?, ?it/s]

Assay  178_684
Total Compounds  953
Inactive Compounds  885
Active Compounds  68
Inactive Compounds (after Undersamplimg)  204
Total Compounds (after Undersamplimg)  272
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 13, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.317810
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 8, 'min_samples_split': 12, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.290626
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 300, 'n_jobs': -1, 

  1%|▍                                        | 1/89 [01:49<2:40:22, 109.34s/it]

Best Threshold=0.240378
Assay  180_686
Total Compounds  597
Inactive Compounds  546
Active Compounds  51
Inactive Compounds (after Undersamplimg)  153
Total Compounds (after Undersamplimg)  204
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.370586
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.368401
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimator

  2%|▉                                        | 2/89 [03:30<2:31:20, 104.37s/it]

...

Best Threshold=0.360470
Assay  283_762
Total Compounds  1544
Inactive Compounds  1437
Active Compounds  107
Inactive Compounds (after Undersamplimg)  321
Total Compounds (after Undersamplimg)  428
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.171836
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.250172
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 

 99%|███████████████████████████████████████▌| 88/89 [4:56:12<02:49, 169.95s/it]

Best Threshold=0.365239
Assay  93_259
Total Compounds  127
Inactive Compounds  93
Active Compounds  34
No Undersamplimg; Assay already in 1:3 ratio or less 127
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.213794
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.363097
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 300, 'n_jobs': -1, 'oob_score'

100%|████████████████████████████████████████| 89/89 [4:57:33<00:00, 200.60s/it]

Best Threshold=0.260921


In [15]:
flatList_master_detail_list = [ item for elem in master_detail_list for item in elem]
list_of_lists_df = pd.DataFrame(flatList_master_detail_list,columns=["assay","InChICode_standardised","fp_proba", "fp_pred", "fp_threshold", "CP_proba", 
                                                     "CP_pred","CP_threshold","true", "ts", "pc", "Data"])
list_of_lists_df["MFP_Correct"] = list_of_lists_df["fp_pred"]==list_of_lists_df["true"]
list_of_lists_df["CP_Correct"] = list_of_lists_df["CP_pred"]==list_of_lists_df["true"]
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.648612,1,0.317810,0.794860,1,0.290626,1.0,0.282609,0.858368,Training,True,True
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.261878,0,0.317810,0.587383,1,0.290626,1.0,0.576923,0.710612,Training,False,True
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.025140,0,0.317810,0.180529,0,0.290626,0.0,0.727273,0.492026,Training,True,True
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.018895,0,0.317810,0.230988,0,0.290626,0.0,0.800000,0.717828,Training,True,True
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.258888,0,0.317810,0.116859,0,0.290626,0.0,0.738462,0.457042,Training,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,"InChI=1S/C8H11ClN2O3S2/c1-4(2)16(13,14)5-3-15-...",0.309551,1,0.240921,0.322350,1,0.260921,0.0,0.250000,0.618373,HeldOut,False,False
58483,93_259,InChI=1S/C18H23NO3/c1-13(12-22-17-6-4-3-5-7-17...,0.267934,1,0.240921,0.223786,0,0.260921,0.0,0.391304,0.603436,HeldOut,False,True
58484,93_259,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.335675,1,0.240921,0.283991,1,0.260921,1.0,0.277778,0.826193,HeldOut,True,True
58485,93_259,InChI=1S/C16H13N3O3/c1-22-16(21)19-15-17-12-8-...,0.334415,1,0.240921,0.371684,1,0.260921,0.0,0.344828,0.763229,HeldOut,False,False


In [16]:
#Probability balancing to thresholds

new_results= pd.DataFrame()
new_new_results = pd.DataFrame()

for t in list(list_of_lists_df["fp_threshold"].unique()):
        
        detail_list_fold=list_of_lists_df[list_of_lists_df["fp_threshold"]==t]
        print(len(detail_list_fold["fp_proba"].values), "Compounds in Fold with Threshold: ", np.round(t, 3))
        #print(detail_list_fold["fp_proba"].values)
        k_list=[]
        for p in detail_list_fold["fp_proba"].values:
            if p<=t:
                #print(p)
                k=p*0.5/t

            else:
                #print(p)
                k=1-((1-p)*0.5)/(1-t)

            k_list.append(k)
        #print(k_list)
        detail_list_fold["fp_proba_scaled"]=k_list
        new_results=pd.concat([new_results, detail_list_fold])
       
for t in list(new_results["CP_threshold"].unique()):
        
        detail_list_fold=new_results[new_results["CP_threshold"]==t]
        print(len(detail_list_fold["fp_proba"].values), "Compounds in Fold with Threshold: ", np.round(t, 3))
        #print(detail_list_fold["fp_proba"].values)
        k_list=[]
        for p in detail_list_fold["CP_proba"].values:
            if p<=t:
                #print(p)
                k=p*0.5/t

            else:
                #print(p)
                k=1-((1-p)*0.5)/(1-t)

            k_list.append(k)
        #print(k_list)
        detail_list_fold["CP_proba_scaled"]=k_list    
        
        new_new_results=pd.concat([new_new_results, detail_list_fold])

new_new_results

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


44 Compounds in Fold with Threshold:  0.318
44 Compounds in Fold with Threshold:  0.212
43 Compounds in Fold with Threshold:  0.447
43 Compounds in Fold with Threshold:  0.295
43 Compounds in Fold with Threshold:  0.32
55 Compounds in Fold with Threshold:  0.325
33 Compounds in Fold with Threshold:  0.371
33 Compounds in Fold with Threshold:  0.484
33 Compounds in Fold with Threshold:  0.326
32 Compounds in Fold with Threshold:  0.216
32 Compounds in Fold with Threshold:  0.353
41 Compounds in Fold with Threshold:  0.289
29 Compounds in Fold with Threshold:  0.314
29 Compounds in Fold with Threshold:  0.283
29 Compounds in Fold with Threshold:  0.623
29 Compounds in Fold with Threshold:  0.339
28 Compounds in Fold with Threshold:  0.477
36 Compounds in Fold with Threshold:  0.256
50 Compounds in Fold with Threshold:  0.334
50 Compounds in Fold with Threshold:  0.278
50 Compounds in Fold with Threshold:  0.271
50 Compounds in Fold with Threshold:  0.213
49 Compounds in Fold with Thresho

256 Compounds in Fold with Threshold:  0.337
27 Compounds in Fold with Threshold:  0.271
26 Compounds in Fold with Threshold:  0.306
26 Compounds in Fold with Threshold:  0.271
26 Compounds in Fold with Threshold:  0.215
26 Compounds in Fold with Threshold:  0.253
33 Compounds in Fold with Threshold:  0.478
180 Compounds in Fold with Threshold:  0.275
179 Compounds in Fold with Threshold:  0.305
179 Compounds in Fold with Threshold:  0.305
179 Compounds in Fold with Threshold:  0.262
179 Compounds in Fold with Threshold:  0.291
224 Compounds in Fold with Threshold:  0.304
98 Compounds in Fold with Threshold:  0.265
98 Compounds in Fold with Threshold:  0.303
98 Compounds in Fold with Threshold:  0.271
98 Compounds in Fold with Threshold:  0.25
97 Compounds in Fold with Threshold:  0.242
123 Compounds in Fold with Threshold:  0.258
48 Compounds in Fold with Threshold:  0.324
47 Compounds in Fold with Threshold:  0.274
47 Compounds in Fold with Threshold:  0.156
47 Compounds in Fold with

490 Compounds in Fold with Threshold:  0.273
490 Compounds in Fold with Threshold:  0.301
613 Compounds in Fold with Threshold:  0.321
112 Compounds in Fold with Threshold:  0.253
111 Compounds in Fold with Threshold:  0.225
111 Compounds in Fold with Threshold:  0.205
111 Compounds in Fold with Threshold:  0.37
111 Compounds in Fold with Threshold:  0.223
140 Compounds in Fold with Threshold:  0.173
146 Compounds in Fold with Threshold:  0.273
145 Compounds in Fold with Threshold:  0.196
145 Compounds in Fold with Threshold:  0.288
145 Compounds in Fold with Threshold:  0.213
145 Compounds in Fold with Threshold:  0.273
182 Compounds in Fold with Threshold:  0.337
254 Compounds in Fold with Threshold:  0.314
254 Compounds in Fold with Threshold:  0.302
254 Compounds in Fold with Threshold:  0.267
254 Compounds in Fold with Threshold:  0.322
254 Compounds in Fold with Threshold:  0.369
318 Compounds in Fold with Threshold:  0.293
388 Compounds in Fold with Threshold:  0.301
388 Compoun

19 Compounds in Fold with Threshold:  0.214
25 Compounds in Fold with Threshold:  0.177
62 Compounds in Fold with Threshold:  0.212
62 Compounds in Fold with Threshold:  0.184
61 Compounds in Fold with Threshold:  0.252
61 Compounds in Fold with Threshold:  0.234
61 Compounds in Fold with Threshold:  0.243
77 Compounds in Fold with Threshold:  0.225
77 Compounds in Fold with Threshold:  0.352
77 Compounds in Fold with Threshold:  0.198
77 Compounds in Fold with Threshold:  0.317
77 Compounds in Fold with Threshold:  0.318
76 Compounds in Fold with Threshold:  0.269
96 Compounds in Fold with Threshold:  0.333
20 Compounds in Fold with Threshold:  0.123
19 Compounds in Fold with Threshold:  0.382
19 Compounds in Fold with Threshold:  0.231
19 Compounds in Fold with Threshold:  0.255
19 Compounds in Fold with Threshold:  0.358
24 Compounds in Fold with Threshold:  0.083
71 Compounds in Fold with Threshold:  0.448
71 Compounds in Fold with Threshold:  0.43
71 Compounds in Fold with Thresho

29 Compounds in Fold with Threshold:  0.191
29 Compounds in Fold with Threshold:  0.326
29 Compounds in Fold with Threshold:  0.252
29 Compounds in Fold with Threshold:  0.389
28 Compounds in Fold with Threshold:  0.271
36 Compounds in Fold with Threshold:  0.389
58 Compounds in Fold with Threshold:  0.294
58 Compounds in Fold with Threshold:  0.453
58 Compounds in Fold with Threshold:  0.37
57 Compounds in Fold with Threshold:  0.297
57 Compounds in Fold with Threshold:  0.329
72 Compounds in Fold with Threshold:  0.334
51 Compounds in Fold with Threshold:  0.311
51 Compounds in Fold with Threshold:  0.405
50 Compounds in Fold with Threshold:  0.269
50 Compounds in Fold with Threshold:  0.381
50 Compounds in Fold with Threshold:  0.238
64 Compounds in Fold with Threshold:  0.235
126 Compounds in Fold with Threshold:  0.266
126 Compounds in Fold with Threshold:  0.311
125 Compounds in Fold with Threshold:  0.335
125 Compounds in Fold with Threshold:  0.246
125 Compounds in Fold with Th

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.648612,1,0.317810,0.794860,1,0.290626,1.0,0.282609,0.858368,Training,True,True,0.742456,0.855408
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.261878,0,0.317810,0.587383,1,0.290626,1.0,0.576923,0.710612,Training,False,True,0.412004,0.709168
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.025140,0,0.317810,0.180529,0,0.290626,0.0,0.727273,0.492026,Training,True,True,0.039553,0.310587
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.018895,0,0.317810,0.230988,0,0.290626,0.0,0.800000,0.717828,Training,True,True,0.029727,0.397398
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.258888,0,0.317810,0.116859,0,0.290626,0.0,0.738462,0.457042,Training,True,True,0.407301,0.201047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,"InChI=1S/C8H11ClN2O3S2/c1-4(2)16(13,14)5-3-15-...",0.309551,1,0.240921,0.322350,1,0.260921,0.0,0.250000,0.618373,HeldOut,False,False,0.545206,0.541558
58483,93_259,InChI=1S/C18H23NO3/c1-13(12-22-17-6-4-3-5-7-17...,0.267934,1,0.240921,0.223786,0,0.260921,0.0,0.391304,0.603436,HeldOut,False,True,0.517793,0.428839
58484,93_259,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.335675,1,0.240921,0.283991,1,0.260921,1.0,0.277778,0.826193,HeldOut,True,True,0.562413,0.515607
58485,93_259,InChI=1S/C16H13N3O3/c1-22-16(21)19-15-17-12-8-...,0.334415,1,0.240921,0.371684,1,0.260921,0.0,0.344828,0.763229,HeldOut,False,False,0.561583,0.574934


In [17]:
new_new_results.to_csv("Predictions_train_heldout_scaled_prob_all_assays.csv", index=False)